# Data Cleaning

All the modifications to the data set except for type conversion and column
renames go here

To do: If you end up adding pdays to the model, you need to do better null handling than setting to 999's


In [1]:
%load_ext sql

In [2]:
%sql postgresql://localhost/bankcalls

'Connected: @bankcalls'

# Feature engineering plan

1. Campagin -> Turn everything 7 or more to a 6 and have a category for 6 or more
2. Previous -> before this campaign, for this client
    Bucket this as 3 or more is 3
    
(See EDA output)

In [3]:
%sql UPDATE bank_addl SET campaign = 6 WHERE campaign > 6; 

 * postgresql://localhost/bankcalls
2406 rows affected.


[]

In [4]:
%sql UPDATE bank_addl SET previous = 3 WHERE previous > 3;

 * postgresql://localhost/bankcalls
94 rows affected.


[]

In [5]:
%sql COMMIT;

 * postgresql://localhost/bankcalls
Done.


[]